In [ ]:
import json

import numpy as np
from web3 import Web3, HTTPProvider

from client import BinanceGateway
import contracts
from contracts import convert_balance_2_units
import utils

"""
Aggregate net risk of LP pool + future positions + deposits (liabilities) to systematically
manage risk of LP pool IL. Main risk; futures gross-up margin risk on coin with large IL, and funding fee
exposure on that position (+ or -)

Pool risk:
  a) get total supply of pool tokens
  b) get our supply of pool tokens
  c) get our % share of pool, and % risk of coins
(note: example does line-by-line but can be done recursively for any pool)

"""

In [ ]:
######################### Config
config = utils.load_config(name="config.ini")
ETHERSCAN_API = json.loads(config.get("API", "ETHERSCAN"))
RISK_LIMIT_USD_PER_COIN = json.loads(config.get("RISK", "RISK_LIMIT_USD_PER_COIN"))
MAX_SLIPPAGE_PCT = json.loads(config.get("RISK", "MAX_SLIPPAGE_PCT"))
MAX_ORDER_SIZE_USD = json.loads(config.get("RISK", "MAX_ORDER_SIZE_USD"))
DEPOSIT_2_POOL = json.loads(config.get("RISK", "DEPOSIT_2_POOL"))

In [ ]:
# factories
polygonFactory = contracts.ContractFactory("polygon")

# init contracts
CONTRACT_POLYGON = {k: polygonFactory.create_contract(*v) for k, v in contracts.ADDRESS_POLYGON.items()}

In [ ]:
def get_pool_composition(pool_contract):
    """
    pool_contract; Web3._utils.datatypes.Contract type
    
    """
    pool = {}
    n = 0
    while True:
        try:
            # get coin contract and balance of LP pool
            coin_address = pool_contract.functions.coins(n).call()
            coin_balance = pool_contract.functions.balances(n).call()
            pool[coin_address] = convert_balance_2_units(coin_balance, unit="ether")        
        except:
            break
        n += 1
        
    return pool


### <font color='blue'> <POOL_NAME> (example; tripled nested meta pool) </font>

In [ ]:
########### inspect POOL_1
# 0 index = <TOKEN>
# 1 index = <LP_TOKEN>
risk_token_1 = convert_balance_2_units(CONTRACT_POLYGON['<POOL_NAME>'].functions.balances(0).call())
lp_token_1 = convert_balance_2_units(CONTRACT_POLYGON['<POOL_NAME>'].functions.balances(1).call())

########### inspect POOL_2
# and what are there balances?
lp2_token_1 = convert_balance_2_units(CONTRACT_POLYGON['<POOL_NAME_2>'].functions.balances(0).call())
lp2_token_2 = convert_balance_2_units(balance=CONTRACT_POLYGON['<POOL_NAME_2>'].functions.balances(1).call(), unit='gwei') * 10
lp2_token_3 = convert_balance_2_units(CONTRACT_POLYGON['<POOL_NAME_2'].functions.balances(2).call())

# what % of POOL_2 is in POOL_1?
lp_token_1_total = convert_balance_2_units(CONTRACT_POLYGON['<POOL_TOKEN_2>'].functions.totalSupply().call())
lp_token_1_pct = lp_token_1 / lp_token_1_total
lp_token_1_pct * lp2_token_2
lp_token_1_pct * lp2_token_3

########### inspect POOL_3
dai = convert_balance_2_units(CONTRACT_POLYGON['<POOL_NAME_3>'].functions.balances(0).call())
usdc = convert_balance_2_units(CONTRACT_POLYGON['<POOL_NAME_3>'].functions.balances(1).call(), unit="mwei")
usdt = convert_balance_2_units(CONTRACT_POLYGON['<POOL_NAME_3>'].functions.balances(2).call(), unit="mwei")

# what % of POOL_3 is in POOL_2?
pool3_risk_total = convert_balance_2_units(CONTRACT_POLYGON['<POOL_TOKEN_3>'].functions.totalSupply().call())
pool3_risk_pct = lp2_token_1 / pool3_risk_total
stables_risk = (dai + usdc + usdt) * pool3_risk_pct * lp_token_1_pct

# Total native risk in pool
risk_pool = {
    "USD": stables_risk,
    "TOKEN_1": risk_token_1,
    "TOKEN_2": lp_token_1_pct * lp2_token_2,
    "TOKEN_3": lp_token_1_pct * lp2_token_3,

}
display(risk_pool)

### <font color='blue'> Risk </font>

In [ ]:
TARGET_ADDRESS = ""
bg = BinanceGateway("BINANCE_FUTURES_USDM", key, secret)

In [ ]:
def get_our_pct_of_pool_risk(pool_name, risk_pool_total):
    """
    pool_name: str
        ie tricrypto
        
    risk_pool_total: dictionary
        total risk of pool, ie {'BTC': 1, 'ETH': 100}
    
    """
    # Our share of the pool
    total_supply_lp = CONTRACT_POLYGON[f'{pool_name}_token'].functions.totalSupply().call()
    our_supply_lp = CONTRACT_POLYGON[f'{pool_name}_gauge'].functions.balanceOf(TARGET_ADDRESS).call()
    our_pool_pct = our_supply_lp / total_supply_lp
    our_risk = {k: v * our_pool_pct for k, v in risk_pool_total.items()}
    return our_risk

# pool positions
our_pool_risk = get_our_pct_of_pool_risk('<POOL_NAME>', risk_pool)

In [ ]:
# pool positions
our_pool_risk

# binance hedges
futs_risk = bg.get_open_futs_positions(ignore_usdt_in_key=True)

# net out risk
risk_mappings = [
    our_pool_risk,
    futs_risk,
    DEPOSIT_2_POOL,
]

display(risk_mappings)

In [ ]:
# net risk across pool, futs, deposit (liabilities)
symb_2_net_native = {}
for d in risk_mappings:
    for symb_, native_ in d.items():
        if symb_ not in symb_2_net_native:
            symb_2_net_native[symb_] = 0
        symb_2_net_native[symb_] += native_ 
symb_2_net_native.pop('USD')
# convert risk to usd
symb_2_spot = bg.get_spot(sorted(symb_2_net_native))
symb_2_net_usd = {k: v * symb_2_spot[k] for k, v in symb_2_net_native.items()}

display(f'net native: {(symb_2_net_native)}')
display(f'net usd: {(symb_2_net_usd)}')

### <font color='blue'> Trade Simple </font>

In [ ]:
def get_weighted_px(risk_dlr, prices: list[list()]):
    """
    risk_dlr;
        how much risk in $ we need to look into order book
    prices;
        order_book['bids'] or order_book['asks']
    
    """
    prices = [[float(px), float(qty)] for px, qty in prices]

    prices_trunc = []
    amount = 0
    for n, (px, qty) in enumerate(prices):
        amount += px * qty
        if amount >= risk_dlr:
            prices_trunc = prices[:n + 1]
            break
    else:
        raise ValueError(f'Not enough liquidity for ${int(risk_dlr)} on {prices}')

    px_trunc = [i[0] for i in prices_trunc]
    qty_trunc = [i[1] for i in prices_trunc]
    px_weighted = np.dot(a=px_trunc, b=qty_trunc) / sum(qty_trunc)
    
    return px_weighted

In [ ]:
# REQUIRED FOR V2 ($500k +)
#   (v2) passive execution --> wait x minutes as maker before taking
#   (v2) define MAX_ORDER_SIZE_USD (based on px weighted)
#   (v2) margin management threshold (if availableBalance < MARGIN_LOWER_THRESH_USD)

In [ ]:
ROUND_PRICE_2_TICK = {
    'BTC': 1,
    'ETH': 2,
}

ROUND_QUANTITY_2_TICK = {
    'BTC': 3,
    'ETH': 3,
}

for symb_, risk_dlr_ in symb_2_net_usd.items():
    
    if abs(risk_dlr_) >= RISK_LIMIT_USD_PER_COIN:
        
        # upate order book
        order_book = bg.get_order_book(f'{symb_}USDT')
        
        ## use tob best
        best_bid = float(order_book['bids'][0][0])
        best_ask = float(order_book['asks'][0][0])
        
        ## use weighted price for our $ size
        # best_weighted_bid = get_weighted_px(risk_dlr_, order_book['bids'])
        # best_weighted_ask = get_weighted_px(risk_dlr_, order_book['asks'])
        
        slippage_ok = best_ask / best_bid - 1 < MAX_SLIPPAGE_PCT[symb_]
        if not(slippage_ok):
            print('slippage too large')
            continue
        
        # set price 
        if risk_dlr_ > 0:
            side = 'sell'
            price = best_bid
        else:
            side = 'buy'
            price = best_ask
            
        # set quantity
        quantity = min(risk_dlr_, MAX_ORDER_SIZE_USD[symb_]) / symb_2_spot[symb_]
        
        # round 2 tick
        quantity = round(quantity, ROUND_QUANTITY_2_TICK[symb_])
        price = round(price, ROUND_PRICE_2_TICK[symb_])
        
        ## cancel any open orders
        ## bg.cancel_open_orders(symbol=f'{symb_}USDT')
        
        
#         bg.post_order(
#             symbol=f'{symb_}USDT',
#             side=side,
#             type_='limit',
#             quantity=quantity,
#             price=price,
#             timeInForce="GTC",
#         )
